# Forecasting Weather using Navier Stokes PDE

In this notebook, we will solve the Navier Stokes PDE thereby predicting the weather pattern at the sea level. 

#### Contents of the Notebook

- [Forecasting Weather using Navier Stokes PDE](#Forecasting-Weather-using-Navier-Stokes-PDE)
    - [Problem Description](#Problem-Description)
    - [Case Setup](#Case-Setup)
    - [Step 1: Creating the geometry](#Step-1:-Creating-the-geometry)
    - [Step 2: Defining the PDEs and creating the nodes.](#Step-2:-Defining-the-PDEs-and-creating-the-nodes.)
    - [Step 3: Setting up the Domain: Assigning the boundary and PDE constraints](#Step-3:-Setting-up-the-Domain:-Assigning-the-boundary-and-PDE-constraints)
    - [Step 4: Adding Inferencer data](#Step-4:-Adding-Inferencer-data)
    - [Step 5: Hydra configuration](#Step-5:-Hydra-configuration)
    - [Step 6: Putting everything together: Solver and training](#Step-6:-Putting-everything-together:-Solver-and-training)
    - [Visualizing the solution](#Visualizing-the-solution)
    - [Conclusion](#Conclusion)

#### Learning Outcomes
- How to use the inbuilt PDE equations present as part of PhysicsNeMo.
- How to scaling and nondimensionalizing techniques to solve the Problem.

## Forecasting Weather using Navier Stokes PDE

### Problem Description

We aim to predict the velocities for 6-hour timesteps using the Navier-Stokes equation. For this, we use the Navier-Stokes equation. The Navier–Stokes equations mathematically express the conservation of momentum and conservation of mass for Newtonian fluids. We will take a 2d projected input from the ERA5 Reanalysis dataset to be used as initial conditions for our input. We will look into the ERA5 dataset in detail in the upcoming notebooks. The process of taking the 3D sphere and projecting it onto a 2D mesh is shown in the diagram below, the 2D mesh is of the size (1440,720)


<center><img src="images/projection.png" alt="Drawing" style="width: 600px;"/></center>

#### Navier-Stokes Equation 
 
The Navier-Stokes equations consist of a time-dependent continuity equation for the conservation of mass, three time-dependent conservation of momentum equations and a time-dependent conservation of energy equation. There are four independent variables in the problem, the $x, y, and z$ spatial coordinates of some domain, and the time $t$. There are six dependent variables; the pressure $p$, density $r$, and temperature $T$, and three components of the velocity vector; the $u$ component is in the $x$ direction, the $v$ component is in the $y$ direction, and the $w$ component is in the $z$ direction, All of the dependent variables are functions of all four independent variables.
 

\begin{equation}
Continuity : \frac{\partial \rho}{\partial t} + \overrightarrow{\nabla}\cdot(\rho\overrightarrow{u})=0 \end{equation}
\begin{equation}
Momentum : \frac{\partial(\rho \overrightarrow{u})}{\partial t} + \overrightarrow{\nabla}\cdot[\rho\overline{\overline{u\otimes u}}] = -\overrightarrow{\nabla p} + \overrightarrow{\nabla}\cdot\overline{\overline{\tau}} + \rho\overrightarrow{f} \end{equation}
\begin{equation}
Energy : \frac{\partial(\rho e)}{\partial t} + \overrightarrow{\nabla}\cdot((\rho e + p)\overrightarrow{u}) = \overrightarrow{\nabla}\cdot(\overline{\overline{\tau}}\cdot\overrightarrow{u}) + \rho\overrightarrow{f}\overrightarrow{u} + \overrightarrow{\nabla}\cdot(\overrightarrow{\dot{q}})+r \end{equation}

We will now use the Navier-Stokes PDE to solve this problem statement to predict the flow using the below stated approximation. 

**Kindly note** : 

A Numerical weather prediction system uses different numerical methods, like the Finite Element method, to solve the following equations: 
- Momentum equations
- Thermodynamic equation
- Moisture equation
- Continuity equation
- Hydrostatic equation

In our case, we use the Navier-Stokes equation, which conserves the following equations: 
- Momentum equations
- Continuity equation

**We follow this approach given the constraints of the bootcamp environment. Hence we do not compare the results with Numerical weather prediction in this case, but we will look into a more robust model going forward in the Data-driven approach and use it to predict the weather forecast and compare with the results from the dataset.**

### Case Setup

Now, let's start solving the problem by importing the required libraries and packages

#### Note : In this notebook we will describe the contents of the [`navier_stokes.py`](../../source_code/navier_stokes/navier_stokes.py) script

```python
import numpy as np
from sympy import Symbol, Eq, Abs, sin, cos

import physicsnemo
from physicsnemo.sym.hydra import to_absolute_path, instantiate_arch, PhysicsNeMoConfig
from physicsnemo.sym.eq.pdes.navier_stokes import NavierStokes
from physicsnemo.sym.geometry.primitives_2d import Rectangle as rect
from physicsnemo.sym.models.fully_connected import FullyConnectedArch
from physicsnemo.sym.key import Key
from physicsnemo.sym.node import Node
from physicsnemo.sym.solver import Solver
from physicsnemo.sym.domain import Domain
from physicsnemo.sym.domain.constraint import (
    PointwiseConstraint,
    PointwiseInteriorConstraint,
)
from physicsnemo.sym.domain.inferencer import PointVTKInferencer
from physicsnemo.sym.utils.io import (
    VTKUniformGrid,
)
```

### Step 1: Creating the geometry

In this problem, we will create the 2-dimensional periodic mesh using `Rectangle` class from the geometry module. We will define the 2-dimensional mesh object using the two end-points. While the input as defined in the problem statement is $(1440,720)$ , we will then pad the y-axis such that we have a periodic geometry, we do this because, we are representing a approximation of the flow in a 2D space by creating a periodic mesh, we try to address the problem to a certain extent by using a periodic 2d mesh. An illustration of the padding can be viewed below: 

<center><img src="images/periodicity_conversion.png" alt="Drawing" style="width: 1000px;"/></center>

```python
    # make geometry for problem
    length = (-0.720, 0.720)
    height  = (-0.720, 0.720)
    box_bounds = {x: length, y: height}

    # define geometry
    rec = rect(
        (length[0], height[0]),
        (length[1], height[1])
    )
```

#### Scaling and Nondimensionalizing the Problem

The input geometry of the problem can be scaled such that the characteristic length is closer to unity and the geometry is centered around origin. Also, it is often advantageous to work with the nondimensionalized form of physical quantities and PDEs. This can be achieved by output scaling, or nondimensionalizing the PDEs itself using some characteristic dimensions and properties. Simple tricks like these can help improve the convergence behavior and can also give more accurate results.

Below we now take the real world parameters that we have and nondimensalise. 

```python
    # Scaling and Nondimensionalizing the Problem
    
    #############
    # Real Params
    #############
    fluid_kinematic_viscosity = 1.655e-5  # m**2/s 
    fluid_density = 1.1614  # kg/m**3
    fluid_specific_heat = 1005  # J/(kg K)
    fluid_conductivity = 0.0261  # W/(m K)

    ################
    # Non dim params for normalisation 
    ################
    # Diameter of Earth : 12742000 m over range of 1.440
    length_scale = 12742000/1.440 
    # 60 hrs = 1 timestep- every inference frame (0.1) is a 6 hour prediction (s)
    time_scale = 60*60*60 
    # Calcuale velocity & pressure scale 
    velocity_scale = length_scale / time_scale  # m/s
    pressure_scale = fluid_density * ((length_scale / time_scale) ** 2)  # kg / (m s**2)
    # Density scale
    density_scale = 1.1614  # kg/m3


    ##############################
    # Nondimensionalization Params for NavierStokes fn
    ##############################
    # fluid params
    nd_fluid_kinematic_viscosity = fluid_kinematic_viscosity / ( length_scale ** 2 / time_scale)
    nd_fluid_density = fluid_density / density_scale
```

We will now use these parameters to scale our velocity and pressure values to train our model.

### Step 2: Defining the PDEs and creating the nodes.

Navier Stokes Equation is available as part of the PhysicsNeMo PDE Toolbox. Let's start the initialization of the equation. Since we are approximating to predict the flow, we will be setting it to a 2-dimensional solver that is time-dependent. We will then use the nondimensionalized parameters from the values calculated above. 

```python
# make navier stokes equations
ns = NavierStokes(nu=nd_fluid_kinematic_viscosity, rho=nd_fluid_density, dim=2, time=True)
```

#### Creating Neural Network nodes

The default `FullyConnectedArch` represents a 6 layer MLP (multi-layer perceptron) architecture with each layer containing 512 nodes and uses swish as the activation function, we will be using a layer size of 256 for our case.

We will define all the code for the problem in the `run` function as shown below. 

```python
@physicsnemo.sym.main(config_path="conf", config_name="config")
def run(cfg: PhysicsNeMoConfig) -> None:

    # make network 
    flow_net = FullyConnectedArch(
        input_keys=[Key("x"), Key("y"), Key("t")],
        output_keys=[Key("u"), Key("v"), Key("p")],
        periodicity={"x": length, "y" : height}, 
        layer_size=256,
    )

    # make nodes to unroll graph on
    nodes = ns.make_nodes() + [flow_net.make_node(name="flow_net")]

```

#### A small note on periodicity: 

As we discussed earlier, since our aim is the predict the flow from a 2d projected input mesh, we can define the periodicity to repeat itself and modify the initial conditions so that the Boundary Constraints need not be enforced, this would set a periodic boundary for the range $(-0.720 , 719)$, this has been set above using the `periodicity` parameter. 

```python
periodicity={"x": length, "y" : height}, 
```

Similarly, we can modify the input initial conditions to allow us to periodically repeat it, this can be done by taking our input image $(720,1440)$ and fill the mesh to create a 2d tile of the dimension $(1440,1440)$ 

<center><img src="images/periodicity_conversion.png" alt="Drawing" style="width: 1200px;"/></center>

### Step 3: Setting up the Domain: Assigning the boundary and PDE constraints



```python
    # make domain add constraints to the solver
    domain = Domain()
```

An L2 loss (default and can be modified) is then constructed from these constraints, which is used by the optimizer to minimize on. Specifying the constraints in this fashion is called soft-constraints.  

$$L = L_{BC} + L_{Residual}$$

**Initial constraints:**

The initial conditions can be sampled using `PointwiseConstraint.from_numpy()` method. This will sample the points for the training given using the `invar` and `outvar` dictionaries provided, going forward we will also read the initial conditions from a `numpy` file, apply the scale transformations and use it for training our model.

The number of points to sample on each constraint is specified using the `batch_size` parameter. 

**Equations to solve:** The Navier-Stokes PDE we defined is enforced on all the points in the interior region. We will use `PointwiseInteriorConstraint` class to sample points in the interior of the geometry. For this problem we have set the `"continuity": 0` ,  `"momentum_x": 0` and  `"momentum_y": 0` for the mesh. The parameter `bounds`, determines the range for sampling the values for variables $x$ and $y$. We will also define a function to load values from numpy. 

```python
def read_wf_data(velocity_scale,pressure_scale):
    path = "/workspace/python/source_code/navier_stokes/data_lat.npy"
    print(path)
    ic = np.load(path).astype(np.float32)
    
    Pa_to_kgm3 = 0.10197
    mesh_y, mesh_x = np.meshgrid(
        np.linspace(-0.720, 0.719, ic[0].shape[0]),
        np.linspace(-0.720, 0.719, ic[0].shape[1]),
        indexing="ij",
    )
    invar = {}
    invar["x"] = np.expand_dims(mesh_x.astype(np.float32).flatten(),axis=-1)
    invar["y"] = np.expand_dims(mesh_y.astype(np.float32).flatten(),axis=-1)
    invar["t"] = np.full_like(invar["x"], 0)
    outvar = {}
    outvar["u"] = np.expand_dims((ic[0]/velocity_scale).flatten(),axis=-1)
    outvar["v"] = np.expand_dims((ic[1]/velocity_scale).flatten(),axis=-1)
    outvar["p"] = np.expand_dims((ic[2]*Pa_to_kgm3/pressure_scale).flatten(),axis=-1)
    
    return invar, outvar
```

Let us now pass the `velocity` and `pressure` scale values obtained from above to scale our inputs and get the `invar` and `outvar` dictionaries for training.

```python
    # make initial condition 
    ic_invar,ic_outvar = read_wf_data(velocity_scale,pressure_scale)
    
    ic = PointwiseConstraint.from_numpy(
            nodes,
            ic_invar,
            ic_outvar,
            batch_size=cfg.batch_size.initial_condition,
            lambda_weighting=lambda_weighting,
        )
    navier.add_constraint(ic, name="ic")
    
    # make interior constraint
    interior = PointwiseInteriorConstraint(
        nodes=nodes,
        geometry=rec,
        outvar={"continuity": 0, "momentum_x": 0, "momentum_y": 0},
        bounds=box_bounds,
        batch_size=cfg.batch_size.interior,
        parameterization=time_range,
    )
    navier.add_constraint(interior, name="interior")
```

### Step 4: Adding Inferencer data

For visualising the Inferred output from the model, we use the `PointVTKInference` class to run inference on our system and save the output to a file for visualising, let us now add it to the domain. 


```python
    # add inference data for time slices
    for i, specific_time in enumerate(np.linspace(0, time_window_size, 10)):
        vtk_obj = VTKUniformGrid(
            bounds=[(-0.720, 0.720), (-0.360, 0.360)],
            npoints=[1440,720],
            export_map={"u": ["u", "v"], "p": ["p"]},
        )
        grid_inference = PointVTKInferencer(
            vtk_obj=vtk_obj,
            nodes=nodes,
            input_vtk_map={"x": "x", "y": "y"},
            output_names=["u", "v", "p"],
            requires_grad=False,
            invar={"t": np.full([720 *1440, 1], specific_time)},
            batch_size=100000,
        )
        navier.add_inferencer(grid_inference, name="time_slice_" + str(i).zfill(4))
```

### Step 5: Hydra configuration

More information on the available configurations can be found in [PhysicsNeMo Configuration](https://docs.nvidia.com/deeplearning/physicsnemo/physicsnemo-sym/user_guide/features/configuration.html).

```yaml
defaults :
  - physicsnemo_default
  - arch:
      - fully_connected
  - scheduler: tf_exponential_lr
  - optimizer: adam
  - loss: sum
  - _self_

save_filetypes : "vtk,npz"

scheduler:
  decay_rate: 0.95
  decay_steps: 3000

training:
  rec_results_freq : 1000
  rec_constraint_freq: 5000
  max_steps : 125000

batch_size:
  initial_condition: 2048
  interior: 2048

```

### Step 6: Putting everything together: Solver and training

 The solver can then be executed using the `solve` method. 

```python
    # make solver
    slv = Solver(cfg, domain)

    # start solver
    slv.solve()


if __name__ == "__main__":
    run()
```

We are now ready to solve the Problem statement !

This example is already saved for you and the code block below executes that script [`navier_stokes.py`](../../source_code/navier_stokes/navier_stokes.py). You are encouraged to open the script and go through the code once before executing. Also, feel free to edit the parameters in the [`config.yaml`](../../source_code/navier_stokes/conf/config.yaml) file of the model and see its effect on the results.

*Given the time and bootcamp constraints in this scenario, we are leveraging a pre-trained approach to optimize our model development process. This allows us to utilize an existing model that has been trained on the dataset earlier, thereby reducing the time and resources needed to train the model from scratch. By adopting this approach, we aim to improve the efficiency and speed of our model development while ensuring that our performance metrics meet the desired criteria. Let us train for 5000 steps, which will take around 5-10 minutes on a A100*

In [ ]:
import os
os.environ["RANK"]="0"
os.environ["WORLD_SIZE"]="1"
os.environ["MASTER_ADDR"]="localhost"

In [ ]:
!python3 ../../source_code/navier_stokes/navier_stokes.py

### Visualizing the solution

Now let's plot the prediction for the following timesteps obtained from neural network solution. A sample script to plot the results is shown below. If the training is complete, you should get the results like shown below. 

<img src="images/u_plot.png" alt="Drawing"/>


**Kindly note:**  Several assumptions were made in this particular problem statement. while the prediction model provides us with results, this is not a real-world representation due to the constraints of the bootcamp environment, but the problem statement was defined in such a way that we employ these approximations to help get the user get comfortable with PhysicsNeMo without getting into the complexities of all the equations needed to design a Numerical weather prediction system. We will be looking at a much realistic weather prediction approach in the Data-driven approach using the FourCastNet model in one of the upcoming notebook.

In [ ]:
import matplotlib
matplotlib.matplotlib_fname()
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np


plt.figure()

network_dir = "./outputs/navier_stokes/inferencers/time_slice_000"

i=0
fig, ax = plt.subplots(nrows=3, ncols=3, figsize=(15,10))

for row in ax:
    for col in row:
        data = np.load(network_dir + str(i) +".npz", allow_pickle=True)
        data = np.atleast_1d(data.f.arr_0)[0]
        x,y,u,v,p = data['x'] , data['y'] , data['u'] , data['v'] , data['p']
        u_r = np.reshape(u, (720, 1440))
        v_r = np.reshape(v, (720, 1440))
        p_r = np.reshape(p, (720, 1440))
        i = i+1
        tmp = col.imshow(u_r,cmap=cm.coolwarm) # Feel free to change from u_r to v_r or p_r to visualise the following parameters

        col.set_title("Time step : "+ str(i))

fig.subplots_adjust(right=0.9)
cbar_ax = fig.add_axes([0.95, 0.15, 0.03, 0.7])
fig.colorbar(tmp, cax=cbar_ax)

plt.show()

## Visualising with ParaView

Let us now visualise the above generated outputs with ParaView. 

Let us now select the files of respective Timesteps as follows: 

Step 1: Head to `jupyter_notebook/navier_stokes/outputs/navier_stokes/inferencers/` to download the respective `.vti` files for visualisation.

<img src="images/paraview_file.png" alt="Drawing"/>

Step 2: Import them into Paraview and use the loop option to loop throgh the results. 

In [ ]:
from IPython.display import Video

Video("images/paraview.webm")

### Conclusion

In the context of our discussion on Physics Informed approaches in PhysicsNeMo, we have reviewed several examples that highlight the significance and potential benefits of incorporating physical principles and constraints in Neural networks.

Furthermore, we would like to inform you that we have provided an optional notebook that addresses the Data assimilation problem discussed in the introduction notebook. This additional resource will further expand on the concepts and techniques involved in assimilating data into physical models and demonstrate how this approach can enhance the accuracy and predictive power of the model. We encourage you to review this optional notebook, as it can provide valuable insights into the practical implementation of Physics Informed approaches in various fields, including engineering, physics, and data science. 

However, we will now shift our focus to data-driven approaches. We will delve into hands-on examples to illustrate how the six-step approach can be used to solve problems using the data-driven approach, which help us to analyze and extract insights from large and complex datasets. By following this approach, we can identify patterns and trends within the data that may not be immediately evident and use this information to develop predictive models.

### Industrial use-case of PhysicsNeMo

While we intend to cover PhysicsNeMo at an introductory level, focusing on key concepts using sample examples, PhysicsNeMo is capable of doing much more, and the applications below showcase its larger potential across various industries.

In [1]:
from IPython.display import IFrame
IFrame("../industry_modulus.pdf", width=1920, height=1200)

--- 

Don't forget to check out additional [Open Hackathons Resources](https://www.openhackathons.org/s/technical-resources) and join our [OpenACC and Hackathons Slack Channel](https://www.openacc.org/community#slack) to share your experience and get more help from the community.

---

# Licensing

Copyright © 2023 OpenACC-Standard.org.  This material is released by OpenACC-Standard.org, in collaboration with NVIDIA Corporation, under the Creative Commons Attribution 4.0 International (CC BY 4.0). These materials may include references to hardware and software developed by other entities; all applicable licensing and copyrights apply.